In [1]:
import pandas as pd
import numpy as np


In [2]:
# Constants:
gpm_to_kgs = 0.0630901964 
users = 373
ns_per_s = 1e9
gpm_m3s = 0.0000630901964
kg_per_lb = 0.453
s_per_interval = 30
s_per_hr = 3600
kwh_per_mwh = 1000

In [3]:
# Importing flow values : 

SitesData = pd.read_csv("../data/site_b.csv")
SitesData['TimeStamp'] = pd.to_datetime(SitesData['TimeStamp'])
assert not SitesData.duplicated(subset=['TimeStamp']).any()
SitesData.set_index('TimeStamp', inplace=True)
SitesData = SitesData.tz_localize('UTC')
SitesData.sort_index(inplace=True)
SitesData = SitesData.resample('30s').mean()
flow_gpm = SitesData['FT1_GPM']
flow_is_nan = flow_gpm.isnull()
flow_gpm.fillna(0., inplace=True)
flow_gpm[flow_gpm < 0] = 0.
flow_m3s = flow_gpm * gpm_m3s
flow_per_user_m3s = flow_m3s / users
flow_per_user_m3s_rolling_mean = flow_per_user_m3s.rolling(600, min_periods=1).mean()
flow_time_s = flow_per_user_m3s_rolling_mean.index.astype(int).values / ns_per_s

cwt_celsius = 5 / 9 * (SitesData['CWT'] - 32)
cwt_celsius[cwt_celsius < 0.] = 0.
cwt_values_celsius = cwt_celsius.resample('1d').median().resample('5min').mean().interpolate()
cwt_values_celsius = cwt_values_celsius.fillna(cwt_celsius.mean())

In [4]:
# Aggregating flow values to match COP 5 min intervals:
flow_values_m3s = flow_per_user_m3s_rolling_mean.resample('5min').sum()

In [5]:
# Calculate heat demand: 
c_p = 4184    #J/kg K
delta_t = (60 - cwt_values_celsius)  # K 
density = 1000 #kg/m3

Q_array = flow_values_m3s*c_p*delta_t*density
Q_array[Q_array == 0] = Q_array.mean()
Q_array = Q_array[0:len(Q_array)-1]

In [6]:
# Calculate Power using heat demand and COP values obtained from TesPy:
COP_site = pd.read_csv("../data/site_b_COP_1d.csv")
COP_site['timestamp_utc'] = pd.to_datetime(COP_site['timestamp_utc'])
COP_site.set_index('timestamp_utc',inplace = True)
COP_site = COP_site.resample('5min').mean().interpolate()
COP_site = COP_site.reindex(Q_array.index).interpolate()

In [7]:
Power_site =  Q_array.values.squeeze()/COP_site['COP']
site_in_kwh = (Power_site * s_per_interval/s_per_hr)/1000

In [8]:
# =============================================================================
# Import and arrange emissions data for the required period
# =============================================================================
Watttime = pd.read_csv("../data/ERCOT_EASTTX-MOER_2021-2023.csv")

# Convert timestamp strings to datetimes
Watttime['timestamp_local'] = pd.to_datetime(Watttime['timestamp_local'])
Watttime['timestamp_utc'] = pd.to_datetime(Watttime['timestamp_utc'])
assert not Watttime.duplicated(subset=['timestamp_utc']).any()
Watttime.set_index('timestamp_utc', inplace=True)
Watttime.sort_index(inplace=True)



In [9]:
Watttime = Watttime.resample('5min').mean().interpolate()

In [10]:
# Get Watttime data corresponding to SitesData time period
Watttime = Watttime[(Watttime.index >= site_in_kwh.index.min()) 
                    & ((Watttime.index <= site_in_kwh.index.max()))]

# Note that this is in pounds
emissions_lbs_per_MWh = Watttime['value']
emissions_kg_per_kWh = emissions_lbs_per_MWh * kg_per_lb / kwh_per_mwh
kwh_and_emissions = (pd.concat([site_in_kwh, emissions_kg_per_kWh], axis=1).dropna()
                     .rename(columns={'COP' : 'kwh', 'value' : 'kg / kwh'}))
daily_emissions_kg = (kwh_and_emissions['kwh'] * kwh_and_emissions['kg / kwh']).resample('d').sum()

In [11]:
daily_emissions_kg.to_csv('../data/site_b_HPWH_emissions_daily.csv')